In [5]:
from tensorflow.keras.models import load_model,Model
from tensorflow.python.keras import backend as k
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from tensorflow.keras.applications import Xception,MobileNetV2
import tensorflow as tf
import tensorflow.keras.layers as layers

from IPython.display import clear_output, Image, display, HTML
import numpy as np
import os

In [2]:
k.set_learning_phase(0)

In [8]:
inputs = tf.placeholder(tf.string, name="input_image", shape=[])
image = tf.image.decode_jpeg(inputs, channels=3)
image = tf.expand_dims(image, 0)
image = tf.image.resize_bilinear(image, [299, 299], align_corners=False)
image = tf.math.divide(image,255.)
image = tf.math.subtract(image,0.5)
image = tf.math.multiply(image,2)

In [2]:
model = Xception(input_tensor=image)

# we can also load weights after compilation 

Instructions for updating:
Colocations handled automatically by placer.


In [18]:
model.load_weights('xception.weights')

In [5]:
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    
show_graph(tf.get_default_graph().as_graph_def())

In [6]:
session = k.get_session()
output = tf.identity(model.output, name='output_node')

In [7]:
prediction_signature = tf.saved_model.signature_def_utils.predict_signature_def({"input": inputs}, {"output":output})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


In [8]:
builder = saved_model_builder.SavedModelBuilder('try_inject_xception/')

In [9]:
builder.add_meta_graph_and_variables(session,
                                     [tag_constants.SERVING],
                                    signature_def_map={"datax":prediction_signature})
builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: try_inject_xception/saved_model.pb


b'try_inject_xception/saved_model.pb'